# Test mass function

This notebook tests the ``cosmo`` class and its routines for the computation of the halo mass function

In [ ]:
import numpy as np
import colibri.cosmology as cc
import matplotlib.pyplot as plt

plt.rc('text',usetex=True)
plt.rc('font',size=15,family='serif')

colors = ['r','b','g','y','m','c','k']

### Pre-define arrays of scales (h/Mpc), redshifts and masses (Msun/h)

In [ ]:
zz   = np.linspace(0., 5., 6)
kk   = np.logspace(-3.,2.,501)
logM = np.arange(7.1,16.,0.1)
nz, nk, nm = len(np.atleast_1d(zz)), len(np.atleast_1d(kk)), len(np.atleast_1d(logM))

### Define a ``cosmo`` instance with default parameters

In [ ]:
C=cc.cosmo()

### Load linear power spectra at requested scales and redshifts

In [ ]:
k,pk=C.camb_Pk(k=kk,z=zz)

### Compute mass variance in spheres

It computes the following quantity

$$
\sigma^2 (M) = \int_0^\infty \frac{\mathrm{d}k \ k^2}{2\pi^2} P_\mathrm{lin}(k) W^2\left[kR(M)\right],
$$

where the window function is set to top-hat by default. Check the documentation to se how to change the window function as well as the field with respect to which the radius-mass relation is computed.
If one wants to use radii instead of masses, it is sufficient to run
```
 M = C.masses_in_radius(R)
 logM = np.log10(M)
```

In [ ]:
sigma_squared = C.mass_variance(logM = logM, k = k, pk = pk)

### Peak height
Peak-background split quantity, i.e. $ \nu = \delta_c/\sigma(M) $, where $\delta_c\approx1.686$ is the linear overdensity for collapse

In [ ]:
nu_peak = C.peak_height(logM = logM, k = k, pk = pk)

### Sheth-Tormen universal function

Sheth-Tormen mass function (takes $\sigma(M)$, previously computed, as argument). Even if its results are functions of $\sigma$, we plot it as function of mass.

In [ ]:
ShethTormen = C.ShethTormen_mass_function(sigma = sigma_squared**0.5, a = 0.707, p = 0.3)

# Halo mass function

Actually, if one is interested in the mass function only, one can skip all the passages above and use the following line.
We use the Sheth-Tormen mass function. If ``'mass_fun' == 'Tinker'`` or ``'MICE'``, one should add ``'z=zz'`` to the arguments of the function.

In [ ]:
HMF = C.halo_mass_function(logM = logM, k = k, pk = pk, window = 'th', mass_fun = 'ShethTormen')

### Plot

In [ ]:
fig, ax = plt.subplots(2,2,figsize=(12,12),sharex=True)
L,R,T,B=0.13,0.96,0.96,0.13
plt.subplots_adjust(left=L,right=R,top=T,bottom=B,wspace=0.25,hspace=0.)


for iz in range(nz):
    # Plot mass variance
    ax[0,0].loglog(10.**logM,sigma_squared[iz],c=colors[iz],label='$z=%i$' %(zz[iz]))

    # Plot peak height
    ax[0,1].loglog(10.**logM,nu_peak[iz],c=colors[iz])

    # Plot Sheth-Tormen
    ax[1,0].loglog(10.**logM,ShethTormen[iz],c=colors[iz])
    ax[1,0].set_ylim(1e-5,1e0)

    # Plot halo mass function
    ax[1,1].loglog(10.**logM,10.**logM*HMF[iz],c=colors[iz])
    ax[1,1].set_ylim(1e-12,1e3)

    # x limits
    ax[0,0].set_xlim(10.**logM.min(),10.**logM.max())

# Labels
for a in [0,1]:
    ax[1,a].set_xlabel('$M \ [M_\odot/h]$')
ax[0,0].set_ylabel('$\sigma^2(M)$')
ax[0,1].set_ylabel('$\\nu(M)=\\frac{\delta_c}{\sigma(M)}$')
ax[1,0].set_ylabel('$\\nu \ f_\mathrm{ST}(\\nu)$')
ax[1,1].set_ylabel('$M \\frac{\mathrm{d}n}{\mathrm{d}M} \ [h^3 \mathrm{Mpc}^{-3}]$')

# Legend and plot
ax[0,0].legend(loc='upper right',fontsize=12, ncol=2)
plt.show()